# Multi-strain modeling

In [ ]:
import pandas
import glob
import cobra

from IPython.display import Image 
import pydotplus
from sklearn.externals.six import StringIO  

from sklearn import tree
import graphviz 

import seaborn

from qbio_resources.multistrain_info import carbon_sources,  strain_classification, phosphorus_sources
%matplotlib inline

----
## A) Simulating growth w/ multi strain models

The goal of this exercise is to provide an introduction to multi-strain metabolic modeling of the E. coli species. The following is largely take from

"Monk, J. M., Charusanti, P., Aziz, R. K., Lerman, J. a, Premyodhin, N., Orth, J. D., … Palsson, B. Ø. (2013). Genome-scale metabolic reconstructions of multiple Escherichia coli strains highlight strain-specific adaptations to nutritional environments. Proceedings of the National Academy of Sciences, 110(50), 20338–20343. http://doi.org/10.1073/pnas.1307797110" 

In the interest of time, only 20 of the models from this study will be used.

### 1) Using 20 models, simulate growth on the 78 carbon containing nutrients in `carbon_sources` and output all predicted growth rates in a dataframe.

The first step in the multi-strain analysis is to quantify the catabolic capabilities of each of the strains in terms of the metabolites that they are capable of growing on.

- The carbon sources are listed below. These are all of the metabolites that show variable growth capabilities among the strains.

- The simulations should take about 3 minutes to run.

- Do not forget to set the lower_bound of glucose uptake to 0 before simulating with the new carbon source AND set the lower bound of the new carbon source to 0 after simulating

In [ ]:
print(carbon_sources)

**Hints:**
  
1) The models are contained in the `multi_strain_models` directory and their filepaths can be listed using `glob`
 - Each model can be loded from the filepath in this list using `cobra.io.load_json_model(filepath)`

In [ ]:
print(glob.glob('../qbio_resources/multi_strain_models/*.json'))

2) The most conventient way to output the results is as a `pandas.DataFrame` (table) 
  - You can then update the values in the table by row (index) and column name using the following

In [ ]:
# Initialize an empty DataFram
test_df = pandas.DataFrame()

# Add new value
test_df.loc['Row 0', 'Column 0'] = 5
test_df.loc['Row 1', 'Column 0'] = 10
test_df.loc['Row 1', 'Column 1'] = 2
test_df

3) When optimizing the model, it is quicker to use `slim_optimize`. This will return only the growth rate of the simulation, not the entire `Solution` instance

In [ ]:
# Load model
model = cobra.io.load_json_model('../qbio_resources/iML1515.json')

# Change carbon source
model.reactions.EX_glc__D_e.lower_bound = 0
model.reactions.EX_ac_e.lower_bound = -10
growth_rate = model.slim_optimize() 
print(growth_rate)

4) Do not forget to set the lower bound of the exchange reaction **back to zero** after simulating!

### 2) Save output of the dataframe as a csv to save progress with `dataframe_name.to_csv(filename)`

### 3) Process the dataframe by removing `nan` values and applying growth threshold
Use 0.1 as the growth rate threshold
- Set dataframe growth rate values less than 0.1 equal to 0
- Set dataframe growth rate values greater than or equal to 0.1 equal to 1

**Hints:**
1) to avoid losing the simulation results during filtering first make a copy of the dataframe like below

In [ ]:
test_filter_df = test_df.copy()

2) The dataframe has nan values if the metabolite/uptake reaction is not present in the model. You can fill nan values to zero with

In [ ]:
test_filter_df.fillna(0, inplace=True)

3) You can edit values in a dataframe based on a filtering criteria (in this case values less than .5) using the following

In [ ]:
test_filter_df[test_filter_df < .5] = 0

### 4) Print the new dataframe to make sure the changes were correctly made

---
## B) Visualizing the results

This section will take the results dataframe from section A and visualize it as a clustered heatmap using `seaborn`

### 1) For visual purposes, transpose the dataframe so that the model IDs are listed in the rows
**Hint:** Dataframe's have a  `dataframe_name.T` property that returns a transposed version of the dictionary.

### 2) Assign colors for each strain based on its type. This will be used to color the rows of the heatmap
Return a dictionary of the form ```strain_color_dict = {model_id: color}```

The collection of strains include the following types:
 1.  **Commensal strains** (lab strains that are not pathogenic)
 2.  **ExPec strains** (Extraintestinal pathogenic strains)
 3.  **InPec strains** (Intestinal pathogenics strains)
 
 
**Hints:**

1) The strain types can be colored according the following dictionary.
 - 'r' corresponds to red
 - 'b' corresponds to blue
 - 'g' corresponds to green

In [ ]:
map_dict = {'Commensal': 'r', 'ExPec': 'b', 'InPec': 'g'}

2) The strain classifications per model id can be found in `strain_classification`, imported above

In [ ]:
print(strain_classification)

3) Dictionaries can be edited like below:

In [ ]:
test_dict = {}
test_dict['key_1'] = 'value_1'
print(test_dict)

### 3) Return a list of colors corresponding to the strain's position in the dataframe's rows (or `index`)
**Hint:** You can map a dictionary to a dataframe's index using the following command and `test_df` from above

In [ ]:
test_dict_for_mapping = {'Row 0': 'value_0', 'Row 1': 'value_1'}
print(test_df.index)
print(test_df.index.map(test_dict_for_mapping))

### 4) Use the `seaborn.clustermap()` function to output the clustered heatmap

- Set `figsize=(15,15)`
- Set `row_colors` equal to the list that was returned in part 3, above

## C) Machine learning to distinguish the strains
Now that we know that the strains do in fact have different growth capabilities. This section will demonstrate how machine learning methods, namely a decision tree, can predict how the strains could be separated based on growth conditions.

### 1) Create numerical classifiers for the strains in the dataframe

Create a list with 0s, 1s, and 2s corresponding to whether a row (`index`) in the dataframe corresponds to a Commensal, ExPec, or InPec strain, respectively.

**Hints:** 
1) You can use the following dictionaries to map the model id contained in the dataframe indexes to the numerical value

In [ ]:
map_dict = {'Commensal': 0, 'ExPec': 1, 'InPec': 2}

In [ ]:
print(strain_classification)

2) You can loop through a dataframes rows (`indexes`) using the following procedure

In [ ]:
for i in test_df.index:
    print(i)

### 2) Initialize a new `tree.DecisionTreeClassifier` instance

- Set the max_depth=3. This determines how many decisions the tree can contain
- Assign the instance to a variable named `classifier`

### 3) Build the decision tree classifier
- This can be executed with `classifier.fit()` where the filtered dataframe from above is the first argument and the list of numerical classifiers from part C1 is the second argument
- Assign the output to a variable named `fit_classifier`

### 4) Pass the fit decision tree classifier and the filtered dataframe into the function below to visualize the tree

In [ ]:
def visualize_classification_tree(fit_classifier, filtered_dataframe):
    dot_data = StringIO()
    model = cobra.io.load_json_model('../qbio_resources/iML1515.json')
    map_dict = {'Commensal': 0, 'ExPec': 1, 'InPec': 2}
    # Get the full names of each of the columns
    feature_names = []
    for i in filtered_dataframe.columns:
        met_id = i.replace('EX_', '')
        if met_id in model.metabolites:
            feature_names.append(model.metabolites.get_by_id(met_id).name)
        else:
            feature_names.append('')
    tree.export_graphviz(fit_classifier, out_file=dot_data,  
                    filled=True, rounded=True,
                    special_characters=True, feature_names=feature_names,
                    class_names = list(map_dict.keys())) 
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
    return Image(graph.create_png())

## D) (If there is time left) Attempt the above analysis with growth capabilities phosphorous sources instead

- The default phosphorus source is phosphate (EX_pi_e). Set this bound to zero before modeling growth with the new phosphorus sources

- The phosphorus sources are:

In [ ]:
print(phosphorus_sources)